In [2]:
import onnx
import torch 
import onnxscript
from transformers import Speech2TextForConditionalGeneration, Speech2TextProcessor

c:\Users\User\dev\SpeechToText\backend\.venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_NAME = "facebook/s2t-small-librispeech-asr"

In [3]:
#--- Converting the encoder:
processor = Speech2TextProcessor.from_pretrained(MODEL_NAME)
model = Speech2TextForConditionalGeneration.from_pretrained(MODEL_NAME)
model.eval()

c:\Users\User\dev\SpeechToText\backend\.venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Speech2TextForConditionalGeneration(
  (model): Speech2TextModel(
    (encoder): Speech2TextEncoder(
      (conv): Conv1dSubsampler(
        (conv_layers): ModuleList(
          (0): Conv1d(80, 1024, kernel_size=(5,), stride=(2,), padding=(2,))
          (1): Conv1d(512, 512, kernel_size=(5,), stride=(2,), padding=(2,))
        )
      )
      (embed_positions): Speech2TextSinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x Speech2TextEncoderLayer(
          (self_attn): Speech2TextAttention(
            (k_proj): Linear(in_features=256, out_features=256, bias=True)
            (v_proj): Linear(in_features=256, out_features=256, bias=True)
            (q_proj): Linear(in_features=256, out_features=256, bias=True)
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=2

In [4]:
encoder = model.model.encoder
dummy_input = torch.randn(1, 500, processor.feature_extractor.feature_size)  # [batch, time, mel_dim]

torch.onnx.export(
    encoder,
    dummy_input,
    "s2t_encoder.onnx",
    opset_version=17,
    input_names=["input_features"],
    output_names=["encoder_outputs"],
    dynamic_axes={
        "input_features": {0: "batch", 1: "time"},
        "encoder_outputs": {0: "batch", 1: "time"},
    }
)

C:\Users\User\AppData\Local\Temp\ipykernel_15072\76168658.py:4: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1122 12:28:29.245000 15072 Lib\site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 17 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W1122 12:28:29.780000 15072 Lib\site-packages\torch\onnx\_internal\exporter\_registration.py:107] torchvision is not installed. Skipping torchvision::nms


[torch.onnx] Obtain model graph for `Speech2TextEncoder([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `Speech2TextEncoder([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 17).
Failed to convert the model to the target version 17 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "c:\Users\User\dev\SpeechToText\backend\.venv\Lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\dev\SpeechToText\backend\.venv\Lib\site-packages\onnxscript\version_converter\_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "c:\Users\User\dev\SpeechToText\backend\.venv\Lib\site-packages\onnxscript\version_converter\__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fil

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 64 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 18},
            producer_name='pytorch',
            producer_version='2.9.1+cpu',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input_features"<FLOAT,[s6,s37,80]>
            ),
            outputs=(
                %"encoder_outputs"<FLOAT,[1,(((s37 - 1)//4)) + 1,256]>
            ),
            initializers=(
                %"conv.conv_layers.1.bias"<FLOAT,[512]>{TorchTensor(...)},
                %"layers.0.self_attn.k_proj.bias"<FLOAT,[256]>{TorchTensor(...)},
                %"layers.0.self_attn.v_proj.bias"<FLOAT,[256]>{TorchTensor(...)},
                %"layers.0.self_attn.q_proj.bias"<FLOAT,[256]>{TorchTensor(...)},
                %"layers.0.self_attn.out_proj.bias"<FLOAT,[256]>{TorchTensor(...)},
                %"layers.0.self_attn_layer_norm.weight"<FLOAT,[256]>{TorchTensor(...

In [1]:
#--- Converting the decoder:

from optimum.onnxruntime import ORTModelForSpeechSeq2Seq
from transformers import Speech2TextProcessor

c:\Users\User\dev\SpeechToText\backend\.venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
 

In [2]:
model_id = "facebook/s2t-small-librispeech-asr"

processor = Speech2TextProcessor.from_pretrained(model_id)

onnx_model = ORTModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    export=True,
    use_cache=False
)

onnx_model.save_pretrained("onnx_s2t")
processor.save_pretrained("onnx_s2t")

Framework not specified. Using pt to export the model.
The task `automatic-speech-recognition` was manually specified, and past key values will not be reused in the decoding. if needed, please pass `--task automatic-speech-recognition-with-past` to export using the past key values.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.
c:\Users\User\dev\SpeechToText\backend\.venv310\lib\site-packages\transformers\configuration_utils.py:448: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}
  warnings.warn(
Using framework PyTorch: 2.1.0+cp

[]